> 언어 모델(Languagel Model) : 단어 시퀀스(문장)에 확률을 할당하는 모델

- 어떤 문장들이 있을 때, 기계가 이 문장은 적절해! 이 문장은 말이 안 돼! 라고 사람처럼 판단할 수 있다면, 기계가 자연어 처리를 정말 잘 한다고 볼 수 있을 겁니다. 이게 바로 언어 모델이 하는 일
- 이번 챕터에서는 통계에 기반한 전통적인 언어 모델(Statistical Languagel Model, SLM)에 대해서 다룰 예정

통계에 기반한 언어 모델은 우리가 실제 사용하는 자연어를 근사하기에는 많은 한계가 있었고, 요즘 들어 인공 신경망이 그러한 한계를 많이 해결해주면서 통계 기반 언어 모델은 많이 사용 용도가 줄었습니다.  
하지만 그럼에도 여전히 통계 기반 언어 모델에서 배우게 될 n-gram은 자연어 처리 분야에서 활발하게 활용되고 있으며, 통계 기반 방법론에 대한 이해는 언어 모델에 대한 전체적인 시야를 갖는 일에 도움이 됩니다.

# 1. 언어 모델(Language Model)이란?
- **언어 모델(Language Model, LM)** : 언어라는 현상을 모델링하고자 단어 시퀀스(또는 문장)에 확률을 할당(assign)하는 모델
    - 통계를 이용한 방법
    - 인공 신경망을 이용한 방법(최근에는 이 쪽이 더 성능 good)[ex: GPT, BERT]

## 1) 언어 모델(Language Model)
- 언어 모델은 가장 자연스러운 단어 시퀀스를 찾아내는 모델
- **단어 시퀀스에 확률을 할당하게 하기 위한 보편적인 방법** : 언어 모델이 이전 단어들이 주어졌을 때 다음 단어를 예측하도록 하는 것(Supervised learning)
- 다른 유형 언어모델 : 주어진 양쪽의 단어들로부터 가운데 비어있는 단어를 예측하는 언어 모델(추론문제) $\Longrightarrow$ 맨 마지막 BERT챕터에서 다룰 내용
- 언어 모델링(Language Modeling) : 언어 모델이 이전 단어들로부터 다음 단어를 예측하는 일

Stanford Univ. 에서는 언어모델링을 grammar로 비유

## 2) 단어 시퀀스의 확률 할당하는 예

#### a. 기계 번역(Machine Translation):  
P(나는 버스를 탔다) > P(나는 버스를 태운다)  
: 언어 모델은 두 문장을 비교하여 좌측의 문장의 확률이 더 높다고 판단합니다.

#### b. 오타 교정(Spell Correction)  
선생님이 교실로 부리나케
P(달려갔다) > P(잘려갔다)  
: 언어 모델은 두 문장을 비교하여 좌측의 문장의 확률이 더 높다고 판단합니다.

#### c. 음성 인식(Speech Recognition)  
P(나는 메롱을 먹는다) < P(나는 메론을 먹는다)  
: 언어 모델은 두 문장을 비교하여 우측의 문장의 확률이 더 높다고 판단합니다.

**언어 모델은 위와 같이 확률을 통해 보다 적절한 문장을 판단합니다.**

## 3) 주어진 이전 단어들로부터 다음 단어 예측하기

#### A. 단어 시퀀스의 확률
하나의 단어를 $w$, 단어 시퀀스을 대문자 $W$라고 한다면, n개의 단어가 등장하는 단어 시퀀스 $W$의 확률은 다음과 같습니다.
P($W$)=P($w_1$,$w_2$,$w_3$,$w_4$,$w_5$,...,$w_n$)

#### B. 다음 단어 등장 확률
이제 다음 단어 등장 확률을 식으로 표현해보겠습니다. n-1개의 단어가 나열된 상태에서 n번째 단어의 확률은 다음과 같습니다.

\begin{align}
P(w_n|w_1,...,w_{n−1})
\end{align}
|의 기호는 조건부 확률(conditional probability)을 의미합니다.

예를 들어 다섯번째 단어의 확률은 아래와 같습니다.  
\begin{align}
P(w_5|w_1,w_2,w_3,w_4)
\end{align}

전체 단어 시퀀스 $W$의 확률은 모든 단어가 예측되고 나서야 알 수 있으므로 단어 시퀀스의 확률은 다음과 같습니다.
\begin{align}
P(W)=P(w_1,w_2,w_3,w_4,w_5,...w_n)=\displaystyle\prod_{i=1}^{n} P(w_n|w_1,...,w_{n−1}) 
\end{align}

## 4) 언어 모델의 간단한 직관
- 비행기를 타려고 공항에 갔는데 지각을 하는 바람에 비행기를 [?]라는 문장이 있습니다. '비행기를' 다음에 어떤 단어가 오게 될지 사람은 쉽게 '놓쳤다'라고 예상할 수 있습니다. 우리 지식에 기반하여 나올 수 있는 여러 단어들을 후보에 놓고 놓쳤다는 단어가 나올 확률이 가장 높다고 판단하였기 때문입니다.

- 그렇다면 기계에게 위 문장을 주고, '비행기를' 다음에 나올 단어를 예측해보라고 한다면 과연 어떻게 최대한 정확히 예측할 수 있을까요? 기계도 비슷합니다. 앞에 어떤 단어들이 나왔는지 고려하여 후보가 될 수 있는 여러 단어들에 대해서 확률을 예측해보고 가장 높은 확률을 가진 단어를 선택합니다. 앞에 어떤 단어들이 나왔는지 고려하여 후보가 될 수 있는 여러 단어들에 대해서 등장 확률을 추정하고 가장 높은 확률을 가진 단어를 선택합니다.

## 5) 검색 엔진에서의 언어 모델의 예

<img src=cap.png width=300>